In [3]:
from utils import TextProcess

In [4]:
embed_size = 128    #Input features to the LSTM
hidden_size = 1024  #Number of LSTM units
num_layers = 1
num_epochs = 20
batch_size = 20
timesteps = 30
learning_rate = 0.002

In [5]:
corpus = TextProcess()

In [ ]:
rep_tensor = corpus.get_data('')